## Install all libraries

In [ ]:
import csv
from bs4 import BeautifulSoup

In [137]:
# Install Firefox & Chrome
!pip install msedge-selenium-tools selenium==3.141

In [1]:
# edge
from msedge.selenium_tools import Edge, EdgeOptions

# # Launch Microsoft Edge (EdgeHTML)
options = EdgeOptions()
options.use_chromium =True
driver = Edge(options=options)

In [2]:
url ='https://www.amazon.com'
driver.get(url)

In [3]:
def get_url(search_term, page):
    """Generate search terms form url"""
    template = 'https://www.amazon.com/s?k={search_term}&qid=1639673296&ref=sr_pg_{page}'
    search_term = search_term.replace(' ','+')
    return template.format(search_term = search_term,page=page)



In [4]:
url = get_url('monitor',2)
driver.get(url)


## Extract the collection

In [ ]:

soup = BeautifulSoup(driver.page_source, 'html.parser')


In [ ]:
results = soup.find_all('div', {"data-component-type": "s-search-result"})
len(results)

22

## Prototype the data results

In [ ]:
# Data need to scrape product_id, product_name, description, price, image, rating,category_id
item = results[0]
aTag = item.h2.a 
aTag.text.strip()

'LG 32QN600-B 32-Inch QHD (2560 x 1440) IPS Monitor with HDR 10, AMD FreeSync with Dual HDMI Inputs, Black'

In [ ]:
product_name = aTag.text 
product_name

'LG 32QN600-B 32-Inch QHD (2560 x 1440) IPS Monitor with HDR 10, AMD FreeSync with Dual HDMI Inputs, Black '

In [ ]:
rating = item.i.text
rating

'4.7 out of 5 stars'

In [ ]:
url = 'https://www.amazon.com' + aTag.get('href')
print(url)

https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A10025863LB87K3EBUFAE&url=%2FLG-32QN600-B-32-Inch-Monitor-FreeSync%2Fdp%2FB088YN9LBK%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dmonitor%26qid%3D1639727042%26sr%3D8-1-spons%26psc%3D1&qualifier=1639727042&id=956761015867833&widgetName=sp_atf


## Get the product data

In [63]:
import csv
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}'
    search_term = search_term.replace(' ', '+')
    
    # Add Term Query To URL
    url = template.format(search_term)
    
    # Add Page Query Placeholder
    url += '&page={}'
    
    return url

def extract_record(item,product_id):
    atag = item.h2.a
    description = 'description no found'
    product_name = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        price = item.find('span', 'a-price').find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
    except AttributeError:
        rating = ''
    
    image = item.find('img', {'class': 's-image'}).get('src')   
    result = (product_id,product_name,description, price, rating,url, image)
    return result


'''Run Main Program Routine'''
def get_product_data(search_term):
    # Initilize product_id
    product_id = 200
    # Startup The Webdriver
    options = EdgeOptions()
    options.use_chromium =True
    driver = Edge(options=options)
    
    records = []
    url =get_url(search_term)
    
    ## Change the number of pages here
    num_page = 1
    for page in range(1, num_page+1):
        driver.get(url.format(page))
        soup =BeautifulSoup(driver.page_source, 'html.parser')
        
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            record = extract_record(item,product_id)
            product_id = product_id + 1
            if record:
                records.append(record)
                
    driver.close()
    # Save Results To CSV File
    with open(f'{search_term}.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['product_id','product_name','description', 'price', 'rating', 'URL', 'image'])
        writer.writerows(records)
    return records
    
    

## Scrape Review from Amazon specific website 

In [64]:
import csv
def get_product_link(search_term):
    urls = []
    with open(f'./{search_term}.csv') as f:
        reader = csv.reader(f)
        for row in reader:
            print(row[5])
            urls.append(row[5])
    del urls[:1]
    return urls

In [53]:
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions

def get_all_reviews_url(urls):
    options = EdgeOptions()
    options.use_chromium =True
    driver = Edge(options=options)
    all_reviews_urls = []
    # use driver to travese through each link
    for url in urls:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        item = soup.find('a', {'data-hook': 'see-all-reviews-link-foot'})
        if item:
            link = 'https://www.amazon.com'+ item.get('href')
            all_reviews_urls.append(link)
        else:
            all_reviews_urls.append("url not found")
    return all_reviews_urls


In [ ]:
all_reviews_urls

['https://www.amazon.com/Unlocked-Smartphone-Intuitive-Warranty-Lavender/product-reviews/B097CMXJ4F/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.com/Samsung-Factory-Unlocked-Smartphone-Resistant/product-reviews/B08XX4P7LB/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.com/Unlocked-Smartphone-Intuitive-Warranty-Lavender/product-reviews/B097CMXJ4F/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.com/Samsung-Electronics-Unlocked-Smartphone-Foldable/product-reviews/B097CNBDX2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.com/Samsung-Factory-Unlocked-Smartphone-Pro-Grade/product-reviews/B08FYTSXGQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.com/Mahogany-Charging-Station-Organizer-Samsung/product-reviews/B00748X51M/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.com/Sams

In [ ]:
all_reviews_urls[0]

'https://www.amazon.com/Unlocked-Smartphone-Intuitive-Warranty-Lavender/product-reviews/B097CMXJ4F/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

## Scrape review data

In [ ]:
## Get review data

In [54]:
import csv
## get a list of product ids
def get_product_ids(search_name):
    product_list = []
    with open(f'./{search_name}.csv') as f:
        reader = csv.reader(f)
        for row in reader:
                product_list.append(row[0])
    product_list.pop(0)
    return product_list

In [55]:
import pandas as pd
import re
import datetime 
import random

In [56]:
def get_current_time():
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    return now

In [58]:
def get_review_on_page(soup, init_id,product_id):
    review_ids = []
    review_time = []
    user_ids = []
    product_ids = []
    temp_id = init_id
    #Get review rating 
    rating = soup.find_all('i',class_='review-rating')
    del rating[:2] ## Delete 2 first unecessary elements 
    overalls = []
    for i in range(0,len(rating)):
        temp_rating = re.search("[+-]?\d+\.\d+", rating[i].getText()).group()
        overalls.append(temp_rating)
        review_ids.append(temp_id)
        temp_id = temp_id + 1
        review_time.append(get_current_time())
        user_ids.append(random.randint(1, 20))
        product_ids.append(product_id)
    review = soup.find_all("span",{"data-hook":"review-body"})
    
    # Get review content
    review_texts = []
    for i in range(0,len(review)):
        review_texts.append(review[i].get_text())
    review_texts[:] = [reviews.lstrip('\n') for reviews in review_texts]
    review_texts[:] = [reviews.rstrip('\n') for reviews in review_texts]
    return review_ids,review_texts, review_time,overalls,user_ids,product_ids, temp_id


In [26]:
    # Write CSV file 
def write_csv(review_ids,review_texts,review_time,overalls,user_ids,product_ids):
 df = pd.DataFrame()
 df['review_id']=pd.Series(review_ids)
 df['review_text']=pd.Series(review_texts)
 df['review_time']=pd.Series(review_time)
 df['overall']=pd.Series(overalls)
 df['user_id']=pd.Series(user_ids)
 df['product_id']=pd.Series(product_ids)
 df.to_csv(f'./samsung phone/review_test.csv',index=False)  
    

In [28]:

def get_product_reviews(urls,search_name):
    ## Run driver 
    options = EdgeOptions()
    options.use_chromium =True
    driver = Edge(options=options)
    
    ## Innitilize data
    init_rvId = 200
    review_ids = []
    review_texts = []
    review_time = []
    overalls = []
    user_ids = []
    product_ids = []
    init_product_ids = get_product_ids(search_name)
    
    for url in urls:
        driver.get(url)
        soup =BeautifulSoup(driver.page_source, 'html.parser')
        result = get_review_on_page(soup, init_rvId,init_product_ids[0])
        review_ids =  review_ids + result[0]
        review_texts = review_texts + result[1]
        review_time = review_time + result[2]
        overalls =  overalls + result[3]
        user_ids = user_ids + result[4]
        product_ids = product_ids + result[5]
        init_rvId = result[6]
        del init_product_ids[:1]
    driver.close()
    
    write_csv(review_ids,review_texts,review_time,overalls,user_ids,product_ids)
    


In [202]:
run(all_reviews_urls,'samsung phone')

## Main program

In [67]:
def main(search_term):
    results = get_product_data(search_term)
    urls = get_product_link(search_term)
    all_review_urls = get_all_reviews_url(urls)
    get_product_reviews(all_review_urls,search_term)
    

In [68]:
main('gaming mouse')

URL
https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0316689GFPTT1BNVDOO&url=%2FMSI-GM41-Lightweight-Fast-Charging-Programmable%2Fdp%2FB094N6J1WP%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dgaming%2Bmouse%26qid%3D1639845464%26sr%3D8-1-spons%26psc%3D1&qualifier=1639845464&id=7508788357805600&widgetName=sp_atf
https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A001535815N3FB8LTONRR&url=%2FCORSAIR-IRONCLAW-Wireless-RGB-Slipstream%2Fdp%2FB07Q424WFW%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dgaming%2Bmouse%26qid%3D1639845464%26sr%3D8-2-spons%26psc%3D1&qualifier=1639845464&id=7508788357805600&widgetName=sp_atf
https://www.amazon.com/Razer-Death-Adder-Essential-RZ01-02540100-R3U1/dp/B07F7T8J9P/ref=sr_1_4?keywords=gaming+mouse&qid=1639845464&sr=8-4
https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_mtf_aps_sr_pg1_1?ie=UTF8&adId=A07188861OOZURHYICUHB&url=%2FFIODIO-Backlight-Adjustable-Comfortable-Ergono